In [2]:
import os
from google.cloud import aiplatform

BUCKET_URI = f"gs://udemy-gcp-mlops" 
PROJECT_ID='dataanalytics-347914'
REGION = "us-central1"
aiplatform.init(project=PROJECT_ID, location=REGION)

In [1]:
display_name = "gsod_unique"

# IMPORT_FILE = "bq://bigquery-public-data.samples.gsod"

# dataset = aiplatform.TabularDataset.create(
#     display_name="NOAA historical weather data_unique",
#     bq_source=[IMPORT_FILE],
# )

# label_column = "mean_temp"

# print(dataset.resource_name)

In [3]:
gcs_source = f"{BUCKET_URI}/gsod_data.csv"

ds = dataset = aiplatform.TabularDataset.create(
    display_name="NOAA historical weather data_unique",
    gcs_source=gcs_source,
)

Creating TabularDataset
Create TabularDataset backing LRO: projects/936546808722/locations/us-central1/datasets/4337283650262401024/operations/5498621538422751232
TabularDataset created. Resource name: projects/936546808722/locations/us-central1/datasets/4337283650262401024
To use this TabularDataset in another session:
ds = aiplatform.TabularDataset('projects/936546808722/locations/us-central1/datasets/4337283650262401024')


In [4]:
TRANSFORMATIONS = [
    {"auto": {"column_name": "year"}},
    {"auto": {"column_name": "month"}},
    {"auto": {"column_name": "day"}},
]

label_column = "mean_temp"

In [5]:
job = aiplatform.AutoMLTabularTrainingJob(
    display_name=display_name,
    optimization_prediction_type="regression",
    optimization_objective="minimize-rmse",
    column_transformations=TRANSFORMATIONS,
)

print(job)

In [6]:
model = job.run(
    dataset=dataset,
    model_display_name=display_name,
    training_fraction_split=0.6,
    validation_fraction_split=0.2,
    test_fraction_split=0.2,
    budget_milli_node_hours=1000,
    disable_early_stopping=False,
    target_column=label_column,
)

View Training:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/2100776497131290624?project=936546808722
AutoMLTabularTrainingJob projects/936546808722/locations/us-central1/trainingPipelines/2100776497131290624 current state:
PipelineState.PIPELINE_STATE_RUNNING
AutoMLTabularTrainingJob projects/936546808722/locations/us-central1/trainingPipelines/2100776497131290624 current state:
PipelineState.PIPELINE_STATE_RUNNING
AutoMLTabularTrainingJob projects/936546808722/locations/us-central1/trainingPipelines/2100776497131290624 current state:
PipelineState.PIPELINE_STATE_RUNNING
AutoMLTabularTrainingJob projects/936546808722/locations/us-central1/trainingPipelines/2100776497131290624 current state:
PipelineState.PIPELINE_STATE_RUNNING
AutoMLTabularTrainingJob projects/936546808722/locations/us-central1/trainingPipelines/2100776497131290624 current state:
PipelineState.PIPELINE_STATE_RUNNING
AutoMLTabularTrainingJob projects/936546808722/locations/us-central1/trai

KeyboardInterrupt: 

In [14]:
MODEL_ID='6073181063323058176'
model = aiplatform.Model('projects/{}/locations/us-central1/models/{}'.format(PROJECT_ID,MODEL_ID))

In [15]:
model_evaluations = model.list_model_evaluations()
eval_res = model_evaluations[0].to_dict()
evaluation_metrics = eval_res["metrics"]

In [16]:
evaluation_metrics

{'rSquared': 0.2674821,
 'rootMeanSquaredLogError': 0.84486413,
 'meanAbsoluteError': 15.600414,
 'meanAbsolutePercentageError': 9897833.0,
 'rootMeanSquaredError': 20.727612}

In [12]:
endpoint = model.deploy(machine_type="n1-standard-4")

Creating Endpoint
Create Endpoint backing LRO: projects/1090925531874/locations/us-central1/endpoints/1474227389545316352/operations/3348446870073507840
Endpoint created. Resource name: projects/1090925531874/locations/us-central1/endpoints/1474227389545316352
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/1090925531874/locations/us-central1/endpoints/1474227389545316352')
Deploying model to Endpoint : projects/1090925531874/locations/us-central1/endpoints/1474227389545316352
Deploy Endpoint model backing LRO: projects/1090925531874/locations/us-central1/endpoints/1474227389545316352/operations/3751519036723167232
Endpoint model deployed. Resource name: projects/1090925531874/locations/us-central1/endpoints/1474227389545316352


In [13]:
INSTANCE = {"year": "1932", "month": "11", "day": "6"}

instances_list = [INSTANCE]

prediction = endpoint.predict(instances_list)
print(prediction)

Prediction(predictions=[{'lower_bound': 11.37252235412598, 'upper_bound': 60.95551300048828, 'value': 45.11862182617188}], deployed_model_id='2542152247278567424', model_version_id='1', model_resource_name='projects/1090925531874/locations/us-central1/models/6073181063323058176', explanations=None)
